# Pet-i Vital Model

이 노트북은 반려견 바이탈 사인(심박수, 호흡수, 체온 등) 데이터를 이용하여 정상(0) vs. 비정상(1) 상태를 분류하는 이진 분류 모델의 전체 과정을 단계별로 실행합니다.

## 1. 환경 설정
필요한 라이브러리를 임포트하고 재현성을 위해 시드를 고정합니다.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 재현성 설정
np.random.seed(42)
tf.random.set_seed(42)


## 2. 데이터 로드 및 합성
실제 CSV 파일을 로드하거나, 파일이 없으면 합성 데이터를 생성합니다.

In [ ]:
data_file = 'pet_vital_data_with_labels.csv'

def generate_dataset(num_normal=5000, num_abnormal=500):
    states = [0, 1, 2]
    dog_types = [0, 1, 2]
    dog_type_probs = [0.3, 0.5, 0.2]

    def gen_state(state, count, abnormal=False):
        df = pd.DataFrame({
            'HeartRate': np.random.randint(50, 200, count),
            'RespirationRate': np.random.randint(5, 50, count),
            'Temperature': np.round(np.random.uniform(36.0, 40.5, count), 1),
            'State': state,
            'DogType': np.random.choice(dog_types, count, p=dog_type_probs)
        })
        if abnormal:
            df['HeartRate'] *= np.random.uniform(1.2, 2.0, count)
            df['RespirationRate'] *= np.random.uniform(1.2, 2.0, count)
            df['Temperature'] += np.random.uniform(0.5, 1.0, count)
        df['Label'] = int(abnormal)
        return df

    normal = pd.concat([gen_state(s, num_normal, False) for s in states])
    abnormal = pd.concat([gen_state(s, num_abnormal, True) for s in states])
    return pd.concat([normal, abnormal]).reset_index(drop=True)

# 로드 또는 생성
if os.path.exists(data_file):
    df = pd.read_csv(data_file)
    print('CSV 로드 완료, 샘플 수 =', len(df))
else:
    df = generate_dataset()
    df.to_csv(data_file, index=False)
    print('합성 데이터 생성 및 저장, 샘플 수 =', len(df))

df.head()

## 3. 전처리 (학습/테스트 분할 및 스케일링)

In [ ]:
# 특징과 레이블 분리
X = df[['HeartRate','RespirationRate','Temperature','State','DogType']]
y = df['Label']

# 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print('Train:', X_train.shape, 'Test:', X_test.shape)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 4. 모델 정의 및 컴파일

In [ ]:
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

## 5. 모델 학습

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('pet_vital_model.keras', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=1024,
    callbacks=callbacks,
    verbose=1
)

## 6. 모델 평가

In [ ]:
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

print('Classification Report:')
print(classification_report(y_test, y_pred, digits=4))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

## 7. 학습 과정 시각화

In [ ]:
plt.figure(figsize=(12,5))
# 정확도
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 손실
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## 8. 새로운 샘플 예측 예시

In [ ]:
# 임의 샘플 생성 함수
def random_vital_sample():
    return np.array([[
        np.random.randint(50,200),
        np.random.randint(5,50),
        round(np.random.uniform(36.0,40.5),1),
        np.random.choice([0,1,2]),
        np.random.choice([0,1,2])
    ]])

samples = np.vstack([random_vital_sample() for _ in range(10)])
samples_scaled = scaler.transform(samples)
probs = model.predict(samples_scaled)
classes = (probs > 0.5).astype(int).reshape(-1)

for i, (prob, cls) in enumerate(zip(probs, classes), 1):
    print(f"샘플 {i}: 확률={prob[0]:.4f}, 예측={cls}\n")